##### Get data
1. Compile HME reviewed data
2. Compile data for evaluation

In [2]:
import os
import re
import pandas as pd

#### Data for eval

In [3]:
df_eval = pd.read_excel(r"..\data\labeled\eval\Bad_Non Bad_all_in_one_updated.xlsx")
print(df_eval.shape)
df_eval[:3]

(770, 5)


,ID,REMARKS,CONTACT_DATE,On or After 2014-04-03,CATEGORY
0,3995608,"""""""NAME CHECK CLEAR""""""",2010-06-05,False,Bad
1,7415322,"""""""LISTED SUBJECT WAS OBSERVED LOITERING WITH ...",2015-06-20,True,Bad
2,4696416,"""""""loitering at an area known for gang and nar...",2011-10-24,False,Bad


In [4]:
df_eval['BAD'] = pd.get_dummies(df_eval['CATEGORY'])['Bad']
# df_eval['Post403'] = df_eval['On or After 2014-04-03'].astype(int)
df_eval2 = df_eval[['ID','REMARKS','BAD']]
df_eval2[:3]

,ID,REMARKS,BAD
0,3995608,"""""""NAME CHECK CLEAR""""""",1
1,7415322,"""""""LISTED SUBJECT WAS OBSERVED LOITERING WITH ...",1
2,4696416,"""""""loitering at an area known for gang and nar...",1


#### HME reviews

In [5]:
os.listdir(r"..\data\labeled\train")

['20170907 - Samples3 (HME Rev 09.14.17).xlsx',
 '20171017 - Summary and Samples (HME Rev).xlsx',
 'Copy of Claro Sample Review.HME.xlsx']

In [6]:
# Muliple tabs
xl1 = pd.ExcelFile(r"..\data\labeled\train\20170907 - Samples3 (HME Rev 09.14.17).xlsx")
xl1.sheet_names

['Summary of Samples',
 'STOP_INNO',
 'LITT',
 'SCHO',
 'CTA',
 'WARR',
 'CANN',
 'MATC_DESC',
 'GAMB',
 'RESP_CALL',
 'PUBL_SELL',
 'PUBL_SLEE',
 'SMOK_FIRE',
 'PUBL_DRIN',
 'DIST_WEAP',
 'PANH']

In [11]:
xl1_data = []
for sheet in xl1.sheet_names[1:]:
    sheet_data = xl1.parse(sheet)
    xl1_data.append(sheet_data[['ID', 'Remarks', 'HME Call']])
    
print(len(xl1_data))
xl1_data[0][:3]

15


,ID,Remarks,HME Call
0,3736142,"""R/O's observed above subject with two other s...",Y
1,3761761,"""LOITERING IN CITGO GAS STATION, FIELD INTERVI...",N
2,3746471,"""SUBJECT, A KNOWN LA RAZA GANG MEMBER, WAS LOI...",Y


In [12]:
df1 = pd.concat(xl1_data,axis=0)
print(df1.shape)
df1[:3]

(2635, 3)


,ID,Remarks,HME Call
0,3736142,"""R/O's observed above subject with two other s...",Y
1,3761761,"""LOITERING IN CITGO GAS STATION, FIELD INTERVI...",N
2,3746471,"""SUBJECT, A KNOWN LA RAZA GANG MEMBER, WAS LOI...",Y


In [13]:
df1['HME Call'].value_counts()

n       1712
y        627
Y        219
N         70
n/y        2
Grey       1
t          1
N          1
Q          1
n          1
Name: HME Call, dtype: int64

In [14]:
df1['BAD'] = df1['HME Call'].map(lambda x:str(x).lower()=='y').astype(int)
df1[:3]

,ID,Remarks,HME Call,BAD
0,3736142,"""R/O's observed above subject with two other s...",Y,1
1,3761761,"""LOITERING IN CITGO GAS STATION, FIELD INTERVI...",N,0
2,3746471,"""SUBJECT, A KNOWN LA RAZA GANG MEMBER, WAS LOI...",Y,1


In [15]:
xl2 = pd.ExcelFile(r"..\data\labeled\train\20171017 - Summary and Samples (HME Rev).xlsx")
xl2.sheet_names

['Summary of Bad Stop Categories',
 'Samples - Bad Stop',
 'Samples - Not Bad Stop']

In [16]:
xl2_data = []
for sheet in xl2.sheet_names[1:]:
    sheet_data = xl2.parse(sheet)
#     print(sheet_data.columns)
    xl2_data.append(sheet_data[['ID', 'REMARKS', 'HME CALL']])

print(len(xl2_data))
xl2_data[0][:3]    

2


,ID,REMARKS,HME CALL
0,6314518,"Assisted in stop of suspisious persons, subjec...",Citizen contact
1,4913097,above subject seen causing a dsturbance on str...,Good stop—causing disturbance
2,3773334,ON SCENE ON STREET DIS,"Depends on date, but could be a citizen contact"


In [17]:
df2 = pd.concat(xl2_data,axis=0)
print(df2.shape)
df2['HME CALL'].value_counts()

(770, 3)


n                                                             363
y                                                             310
Good stop                                                      28
Not a stop                                                      5
Victim—not a stop                                               2
Doesn't seem like a stop                                        2
Citizen contact                                                 2
Good stop. Sounds like trespassing.                             2
Citizen contact / vehicle passenger                             2
Grey area, so let's exclude it                                  2
Good stop—Matched description + robbery should be excluded      1
Not a stop. Wellness check.                                     1
Good stop—causing disturbance                                   1
Good stop—Probably a curfew stop                                1
Let's exclude it. Hard to tell if it's even a stop.             1
Depends on

In [18]:
df2['BAD'] = df2['HME CALL'].map(lambda x: str(x).lower()=="y").astype(int)
df2[:3]

,ID,REMARKS,HME CALL,BAD
0,6314518,"Assisted in stop of suspisious persons, subjec...",Citizen contact,0
1,4913097,above subject seen causing a dsturbance on str...,Good stop—causing disturbance,0
2,3773334,ON SCENE ON STREET DIS,"Depends on date, but could be a citizen contact",0


In [19]:
xl3 = pd.ExcelFile(r"..\data\labeled\train\Copy of Claro Sample Review.HME.xlsx")
xl3.sheet_names

['WORKSHEET', 'SUMMARY']

In [20]:
df3 = xl3.parse(xl3.sheet_names[0])
print(df3.shape)
df3[:3]

(1535, 5)


,SOURCE,CLARO_CALL,HME_CALL,ID,REMARKS
0,Sample 1,y,y,5048152,NaN
1,Sample 1,y,Q - disturbance / involved,4342287,"""PARTY TO A DISTURBANCE"""
2,Sample 1,y,y,5841665,"""subject loitering on property, name check clear"""


In [21]:
df3['HME_CALL'].value_counts()

Y                                            662
y                                            271
N                                            118
n                                             64
Traffic                                       46
Q - disturbance / involved                    42
Q - private ppty                              36
Y - if post 4/14                              36
Y - dispersal                                 31
traffic                                       25
N - CTA                                       21
Q - disturbance / unclear                     18
Q - match descr                               16
Y - disturbance / present (if after 4/14)     13
N - victim                                    12
Q - curfew                                    12
N - domestic                                  11
Q - panhandling                               10
q - match descr                               10
Q - casing                                     8
N - prostitution    

In [22]:
df3['HME_CALL_N'] = df3['HME_CALL'].map(lambda x:str(x).lower().strip())
df3['HME_CALL_N'].value_counts()

y                                            935
n                                            184
traffic                                       71
q - disturbance / involved                    43
y - if post 4/14                              36
q - private ppty                              36
y - dispersal                                 31
q - match descr                               26
n - cta                                       21
q - disturbance / unclear                     18
n - victim                                    18
n - domestic                                  14
y - disturbance / present (if after 4/14)     13
q - curfew                                    12
q - panhandling                               10
n - witness                                    9
q - casing                                     9
n - prostitution                               7
n - school                                     6
y - casing                                     4
n - assist          

In [23]:
def is_bad(hme_review):
    string = str(hme_review)
    if (re.search("y",string) is not None) & (re.search("4/14", string) is None):
        return 1
    else:
        return 0
print(is_bad("y - disturbance / present (if after 4/14)"), is_bad("y"))

0 1


In [24]:
df3['BAD'] =df3["HME_CALL_N"].map(is_bad)
df3[:3]

,SOURCE,CLARO_CALL,HME_CALL,ID,REMARKS,HME_CALL_N,BAD
0,Sample 1,y,y,5048152,NaN,y,1
1,Sample 1,y,Q - disturbance / involved,4342287,"""PARTY TO A DISTURBANCE""",q - disturbance / involved,0
2,Sample 1,y,y,5841665,"""subject loitering on property, name check clear""",y,1


In [25]:
df1.columns = ['ID','REMARKS','HME CALL','BAD']

In [26]:
train_df = pd.concat([df1[['ID','REMARKS','BAD']],
                      df2[['ID','REMARKS','BAD']],
                      df3[['ID','REMARKS','BAD']]], axis = 0)
print(train_df.shape)
train_df[:3]

(4940, 3)


,ID,REMARKS,BAD
0,3736142,"""R/O's observed above subject with two other s...",1
1,3761761,"""LOITERING IN CITGO GAS STATION, FIELD INTERVI...",0
2,3746471,"""SUBJECT, A KNOWN LA RAZA GANG MEMBER, WAS LOI...",1


#### Add in more data

In [28]:
data_all = pd.read_csv(r"C:/Users/Administrator/Desktop/cpd/data/d823_833.txt",sep="|")
data_all =  data_all[['ID', 'CONTACT_TYPE_CD', 'HOTSPOT_NO', 'SEX_CODE_CD', 'RACE_CODE_CD', 'DISTRICT', 'CONTACT_DATE2']]
print(data_all.shape)
data_all[:3]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3256785, 7)


,ID,CONTACT_TYPE_CD,HOTSPOT_NO,SEX_CODE_CD,RACE_CODE_CD,DISTRICT,CONTACT_DATE2
0,3726079,TRAFFIC,NaN,M,BLK,07,2010-01-01
1,3726080,TRAFFIC,NaN,F,BLK,07,2010-01-01
2,3726035,TRAFFIC,NaN,F,BLK,07,2010-01-01


In [36]:
data_all['TRAFFIC'] = data_all['CONTACT_TYPE_CD'].map(lambda x: str(x.strip()) == 'TRAFFIC').astype(int)
data_all['HOTSPOT'] = data_all['HOTSPOT_NO'].map(lambda x: pd.notnull(x)).astype(int)
data_all['POST403'] = data_all['CONTACT_DATE2'].map(lambda x: x >= "2014-04-03").astype(int)
data_all[:3]

,ID,CONTACT_TYPE_CD,HOTSPOT_NO,SEX_CODE_CD,RACE_CODE_CD,DISTRICT,CONTACT_DATE2,TRAFFIC,HOTSPOT,POST403
0,3726079,TRAFFIC,NaN,M,BLK,07,2010-01-01,1,0,0
1,3726080,TRAFFIC,NaN,F,BLK,07,2010-01-01,1,0,0
2,3726035,TRAFFIC,NaN,F,BLK,07,2010-01-01,1,0,0


In [37]:
train_df_more = pd.merge(train_df, data_all[['ID','SEX_CODE_CD','RACE_CODE_CD','DISTRICT','TRAFFIC','HOTSPOT','POST403']], 
                         how="left", left_on = "ID", right_on = "ID")
eval_df_more  = pd.merge(df_eval2, data_all[['ID','SEX_CODE_CD','RACE_CODE_CD','DISTRICT','TRAFFIC','HOTSPOT','POST403']], 
                         how="left", left_on = "ID", right_on = "ID")
print(train_df_more.shape, eval_df_more.shape)
eval_df_more[:3]

(4940, 9) (770, 9)


,ID,REMARKS,BAD,SEX_CODE_CD,RACE_CODE_CD,DISTRICT,TRAFFIC,HOTSPOT,POST403
0,3995608,"""""""NAME CHECK CLEAR""""""",1,M,BLK,32,1,0,0
1,7415322,"""""""LISTED SUBJECT WAS OBSERVED LOITERING WITH ...",1,M,BLK,67,0,1,1
2,4696416,"""""""loitering at an area known for gang and nar...",1,M,WWH,45,0,0,0
